

# Install necessary dependencies

In [1]:
!pip install -q accelerate -U
!pip install -q -U datasets
!pip install -q scipy
!pip install -q ipywidgets
!pip install -q wandb
!pip install -q transformers
!pip install -q torch
!pip install -q sentencepiece
!pip install -q tqdm
!pip install -q evaluate
!pip install -q rouge_score
!pip install --q tensorflow==2.14


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 3.3 M

 # Import dependencies

In [2]:
# Data libraries
from datasets import load_dataset, Dataset, concatenate_datasets, list_metrics, load_metric

# preprocessing Libraries
import numpy as np
import pandas as pd
from datetime import datetime
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Model libraries
import transformers
import ast
import torch
import sklearn
from transformers import (
    T5Tokenizer, T5TokenizerFast, T5ForConditionalGeneration,
    AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM,
    DefaultDataCollator, T5ForQuestionAnswering, DataCollatorForSeq2Seq, pipeline, AutoModelForQuestionAnswering, BertTokenizer, BertForQuestionAnswering,
    DataCollatorForTokenClassification
)

# Metrics library
import evaluate
from evaluate import evaluator

# Visualization library
import wandb

# Warning libraries
import warnings
warnings.filterwarnings('ignore')

# Utility functions
import t5_utils as utils


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# **Data Visualization**

In [3]:
data_raw = load_dataset('conllpp')
print(data_raw)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})


In [4]:
data_raw['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [5]:
Input= data_raw['train'][0]['tokens']
print(f"text: \n{Input}\n---------------")

print(f"label:\n{data_raw['train'][0]['ner_tags']}")

text: 
['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
---------------
label:
[3, 0, 7, 0, 0, 0, 7, 0, 0]


In [6]:
data_raw['train'].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'pos_tags': Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None),
 'chunk_tags': Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)}

In [7]:
label_names = data_raw['train'].features['ner_tags'].feature.names
print("corresponding name of the lables:\n",label_names)

corresponding name of the lables:
 ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


In [8]:
words = data_raw["train"][0]["tokens"]
labels = data_raw["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

EU    rejects German call to boycott British lamb . 
B-ORG O       B-MISC O    O  O       B-MISC  O    O 


In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [10]:
inputs = tokenizer(data_raw["train"][0]["tokens"], is_split_into_words=True) #is_split_into_words=True as the tokens(input colummns) is sperated in columns
inputs.tokens()

['▁EU',
 '▁reject',
 's',
 '▁German',
 '▁call',
 '▁to',
 '▁boycott',
 '▁British',
 '▁lamb',
 '▁',
 '.',
 '</s>']

In [11]:
inputs.word_ids()

[0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 8, None]

In [12]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [13]:
labels = data_raw["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[3, 0, 7, 0, 0, 0, 7, 0, 0]
[3, 0, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


In [14]:
def tokenize_and_align_labels(examples):
    # examples["tokens"] = ["token classification:" + token for token in examples["tokens"]]
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [15]:
tokenized_datasets_train = data_raw['train'].map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=data_raw["train"].column_names,
)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

In [16]:
tokenized_datasets_validation = data_raw['validation'].map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=data_raw["validation"].column_names,
)

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

In [17]:
tokenized_datasets_validation

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3250
})

In [18]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [19]:
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base", return_dict = True)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [20]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [21]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [28]:
def train(train_data, eval_data, project = "Finetunning-t5", base_model_name = "T5", model = "t5-base",
          max_steps = 100, per_device_train_batch_size = 4, per_device_eval_batch_size = 4, save_steps = 50, eval_steps = 50, logging_steps = 50, checkpoint_saved = 100):
  project = project
  base_model_name = base_model_name
  run_name = base_model_name + "-" + project
  output_dir = "/content/" + run_name
  tokenizer = T5Tokenizer.from_pretrained(model)
  model = T5ForConditionalGeneration.from_pretrained(model)

  trainer = transformers.Seq2SeqTrainer(
      model=model,
      train_dataset=train_data,
      eval_dataset=eval_data,
      # compute_metrics=compute_metrics,
      preprocess_logits_for_metrics= preprocess_logits_for_metrics,
      args=transformers.Seq2SeqTrainingArguments(
          output_dir=output_dir,
          warmup_steps=10,
          per_device_train_batch_size=per_device_train_batch_size,
          per_device_eval_batch_size=per_device_eval_batch_size,
          gradient_accumulation_steps=1,
          gradient_checkpointing=True,
          max_steps=max_steps,
          learning_rate=2.5e-5, # Want a small lr for finetuning
          fp16_full_eval=True,
          logging_steps=logging_steps,              # When to start reporting loss
          logging_dir="./logs",        # Directory for storing logs
          save_strategy="steps",       # Save the model checkpoint every logging step
          save_steps=save_steps,                # Save checkpoints every 50 steps
          evaluation_strategy="steps", # Evaluate the model every logging step
          eval_steps=eval_steps,               # Evaluate and save checkpoints every 50 steps
          do_eval=True,               # Perform evaluation at the end of training
          report_to="wandb",
          run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"          # Name of the W&B run (optional)
      ),
      data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id= -100,
    pad_to_multiple_of=8
),
  )
  model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
  trainer.train()
  model.save_pretrained(f"/content/T5-Finetunning-t5/checkpoint-{checkpoint_saved}")
  tokenizer.save_pretrained(f"/content/T5-Finetunning-t5/checkpoint-{checkpoint_saved}")

In [29]:
train(tokenized_datasets_train, tokenized_datasets_validation, project = "Finetunning-t5_ner", base_model_name = "T5", model = "google/flan-t5-small",
          max_steps = 500, per_device_train_batch_size = 4, per_device_eval_batch_size = 4, save_steps = 50, eval_steps = 50, logging_steps = 50,checkpoint_saved = 50)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Step,Training Loss,Validation Loss
50,34.215300,29.855860
100,27.600500,23.997728
150,22.683400,19.188755
200,17.835200,14.237661
250,13.093000,10.157209
300,10.560600,8.481637
350,9.180700,7.904622
400,8.487700,7.614543
450,7.980500,7.488771
500,7.957100,7.446951


In [30]:
prompt = "EU rejects German call to boycott British lamb . "

In [32]:
# tokenizer = T5Tokenizer.from_pretrained("/content/T5-Finetunning-t5_ner/checkpoint-500")
model = T5ForConditionalGeneration.from_pretrained("/content/T5-Finetunning-t5_ner/checkpoint-500")
model.config.use_cache = False
input_ids = tokenizer(prompt, return_tensors="pt",truncation=True).input_ids
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

,
